<a href="https://colab.research.google.com/github/MatteoRobbiati/notebooks/blob/main/QTI-QML-tutorial/QML_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QTI-TH Forum: a snapshot of Quantum Machine Learning

In [ ]:
# install qibo
!pip install qibo

In [ ]:
# import qibo's packages
import qibo
from qibo import gates, hamiltonians, derivative
from qibo.models import Circuit

# some useful python package
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid', font_scale=1.5)

# to interact with the operating system
import os

# numpy backend is enough for a 1-qubit model
qibo.set_backend('numpy')

### A variational quantum circuit

We are going to use a variational quantum circuit as Machine Learning model.
In order to do this, we will use the circuit's parameters as variational parameters during the train. 

We also need a way to embed some external data into the circuit.

In [ ]:
nqubits = 1
layers = 2

c = Circuit(nqubits)
for q in range(nqubits):
  # an Hadamard gate at the beginning
  c.add(gates.H(q=q))
  # and a sequence of rotation layers as model
  for l in range(layers):
    c.add(gates.RY(q=q, theta=0))
    c.add(gates.RY(q=q, theta=0))
    c.add(gates.RZ(q=q, theta=0))
  c.add(gates.M(0))


This specific sequence of gates is important if we want to implement the following ansatz:

In [ ]:
def inject_parameters(circuit, parameters, x):
  params = []
  index = 0
  
  for l in range(layers):
    # embed the first feature
    params.append(parameters[index] * x[0])
    params.append(parameters[index + 1])
    # embed the second feature
    params.append(parameters[index + 2])
    index += 3

  circuit.set_parameters(params)
  return circuit

In [ ]:
nparams = len(c.get_parameters())
initial_parameters = np.random.randn(nparams) * 5

In [ ]:
# define an hamiltonian
h = hamiltonians.Z(nqubits)

# which can be used passing a quantum state 
dummy_state = np.ones(2 ** nqubits) / np.sqrt(2 ** nqubits)

x = [0.2]

# set them into the circuit together with an x
c = inject_parameters(c, initial_parameters, x)

h.expectation(c.execute(nshots=1000).state())

In [ ]:
class vqregressor:

  def __init__(self, layers, data, labels, nqubits=1):
    """Class constructor."""
    self.nqubits = nqubits
    self.layers = layers
    self.data = data
    self.labels = labels

    # initialize the circuit and extract the number of parameters
    self.circuit = self.ansatz(nqubits, layers)
    print(self.circuit.draw())

    self.nparams = len(self.circuit.get_parameters())
    # set the initial value of the variational parameters
    self.params = np.random.randn(self.nparams)
    self.scale_factors = np.ones(self.nparams)

    self.h = hamiltonians.Z(nqubits)


  def ansatz(self, nqubits, layers):
    """Here we implement the variational model ansatz."""
    c = Circuit(nqubits)
    for q in range(nqubits):
      c.add(gates.H(q=q))
      for l in range(layers):
        c.add(gates.RY(q=q, theta=0))
        c.add(gates.RY(q=q, theta=0))
        c.add(gates.RZ(q=q, theta=0))
    c.add(gates.M(0))

    return c

  def inject_data(self, x):
    """Here we combine x and params in order to perform re-uploading."""
    params = []
    index = 0
    
    for q in range(self.nqubits):
      for l in range(self.layers):
        # embed the first feature
        params.append(self.params[index] * x)
        params.append(self.params[index + 1])
        # embed the second feature
        params.append(self.params[index + 2])
        # update scale factors
        self.scale_factors[index] = x

        index += 3

    self.circuit.set_parameters(params)


  def one_prediction(self, x):
    self.inject_data(x)
    return self.h.expectation(self.circuit.execute().state())


  def predict_sample(self):

    predictions = []
    for i, x in enumerate(self.data):
      predictions.append(self.one_prediction(x))

    return predictions

  # gradient descent
  def gradient_descent(self, learning_rate, epochs):

    # we create a folder
    os.system("mkdir ./live-plotting")
    # ww clean it if already exists
    os.system("rm ./live-plotting/*")


    for epoch in range(epochs):
      dloss, loss = self.evaluate_loss_gradients()
      self.params -= learning_rate * dloss
      print(f'Loss at epoch: {epoch + 1} ', loss)

      self.show_predictions(f'Epoch {epoch +1}', save=True)



  def circuit_derivative(self):
    dcirc = np.zeros(self.nparams)   
    
    for par in range(self.nparams):
      dcirc[par] = qibo.derivative.parameter_shift(
          circuit = self.circuit, 
          hamiltonian = self.h, 
          parameter_index = par, 
          scale_factor = self.scale_factors[par]
          )
    
    return dcirc


  def evaluate_loss_gradients(self):

    dloss = np.zeros(self.nparams)
    loss = 0

    for x, y in zip(self.data, self.labels):
      prediction = self.one_prediction(x)
      mse = (prediction - y)
      loss += mse**2
      dcirc = self.circuit_derivative()
      dloss += 2 * mse * dcirc

    return dloss, loss/len(self.data)


  def show_predictions(self, title, save=False):

    y = np.sin(2*self.data)
    predictions = self.predict_sample()

    plt.figure(figsize=(12,8))
    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.scatter(self.data, y, color='orange', alpha=0.6, label='Original', s=70, marker='o')
    plt.scatter(self.data, predictions, color='purple', alpha=0.6, label='Predictions', s=70, marker='o')

    plt.legend()

    if save:
      plt.savefig(f'./live-plotting/'+str(title)+'.png')
      plt.close()

    plt.show()

In [ ]:
ndata = 30
data = np.random.uniform(-1, 1, ndata)
labels = np.sin(2*data)

In [ ]:
VQR = vqregressor(layers=2, data=data, labels=labels)

In [ ]:
VQR.show_predictions('Without training')

In [ ]:
epochs = 100
VQR.gradient_descent(learning_rate=1e-2, epochs=epochs)

In [ ]:
VQR.show_predictions('After training')

In [ ]:
from PIL import Image

images = []

for epoch in range(epochs):
  images.append(Image.open("./live-plotting/Epoch " + str(epoch + 1) + ".png"))

first_image = images[0]
first_image.save("./training.gif", format="GIF", append_images=images,
               save_all=True, duration=100, loop=0)